# Transformations in Time Series Forecasting

This tutorial introduces time series transformations in sktime for preprocessing and feature engineering.

**Duration:** ~10 minutes

## Learning objectives

By the end of this tutorial, you will be able to:
- Understand the motivation for time series transformations
- Apply target variable transformations (differencing, detrending)
- Create features using FourierFeatures and Holidays
- Combine multiple transformations effectively

## 1. Context and Motivation

Time series transformations serve several purposes:
- **Stationarity**: Making the series stationary for better modeling
- **Feature Engineering**: Creating informative features
- **Normalization**: Scaling data for certain algorithms
- **Noise Reduction**: Smoothing irregular patterns

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sktime.datasets import load_airline
from sktime.forecasting.naive import NaiveForecaster
from sktime.performance_metrics.forecasting import mean_absolute_percentage_error
from sktime.utils.plotting import plot_series

# Load airline dataset for demonstration
y = load_airline()
print(f"Dataset: {y.shape[0]} observations from {y.index[0]} to {y.index[-1]}")

# Plot the original series
plot_series(y, title="Original Airline Passengers Data")
plt.show()

# Split for later evaluation
y_train = y.iloc[:-12]
y_test = y.iloc[-12:]

## 2. Target Variable Transformations

### 2.1 Differencing

Differencing removes trends and can help achieve stationarity.

In [ ]:
from sktime.transformations.series.difference import Differencer

# Create differencing transformer
differencer = Differencer(lags=1)  # First difference
differencer_seasonal = Differencer(lags=12)  # Seasonal difference

# Apply transformations
y_diff = differencer.fit_transform(y_train)
y_seasonal_diff = differencer_seasonal.fit_transform(y_train)

# Plot comparisons
fig, axes = plt.subplots(3, 1, figsize=(12, 10))

plot_series(y_train, ax=axes[0], title="Original Series")
plot_series(y_diff, ax=axes[1], title="First Differenced Series")
plot_series(y_seasonal_diff, ax=axes[2], title="Seasonally Differenced Series")

plt.tight_layout()
plt.show()

print(f"Original series variance: {y_train.var():.2f}")
print(f"First differenced variance: {y_diff.var():.2f}")
print(f"Seasonally differenced variance: {y_seasonal_diff.var():.2f}")

### 2.2 Detrending

Detrending removes linear or polynomial trends from the data.

In [ ]:
from sktime.transformations.series.detrend import Detrender

# Create detrending transformers
linear_detrend = Detrender(forecaster=None)  # Linear detrending

# Apply detrending
y_detrended = linear_detrend.fit_transform(y_train)

# Plot comparison
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

plot_series(y_train, ax=axes[0], title="Original Series")
plot_series(y_detrended, ax=axes[1], title="Detrended Series")

plt.tight_layout()
plt.show()

print(f"Original series mean: {y_train.mean():.2f}")
print(f"Detrended series mean: {y_detrended.mean():.2f}")

## 3. Feature Engineering Transformations

### 3.1 Fourier Features

Fourier features capture periodic patterns in the data.

In [ ]:
from sktime.transformations.series.fourier import FourierFeatures

# Create Fourier features for seasonality
fourier_transformer = FourierFeatures(
    sp_list=[12],  # 12-month seasonality
    fourier_terms_list=[3],  # 3 Fourier terms
)

# Fit and transform
X_fourier = fourier_transformer.fit_transform(y_train)

print(f"Fourier features shape: {X_fourier.shape}")
print(f"Feature names: {X_fourier.columns.tolist()}")
print(X_fourier.head())

# Plot some Fourier features
fig, axes = plt.subplots(2, 2, figsize=(15, 8))
axes = axes.ravel()

for i, col in enumerate(X_fourier.columns[:4]):
    plot_series(X_fourier[col], ax=axes[i], title=f"{col}")

plt.tight_layout()
plt.show()

### 3.2 Holiday Features

Holiday features can capture the impact of special days on time series.

In [ ]:
# Note: Holiday features require additional setup and may not be available in all environments
# This is a demonstration of the concept

try:
    from sktime.transformations.series.holiday import HolidayFeatures

    # Create holiday features
    holiday_transformer = HolidayFeatures(
        country_code="US",  # US holidays
        return_dummies=True,
    )

    # Create a daily series for better holiday demonstration
    daily_index = pd.date_range("2020-01-01", "2021-12-31", freq="D")
    y_daily = pd.Series(np.random.randn(len(daily_index)) + 100, index=daily_index)

    X_holidays = holiday_transformer.fit_transform(y_daily)

    print(f"Holiday features shape: {X_holidays.shape}")
    print(f"Holiday columns: {X_holidays.columns.tolist()[:10]}...")  # Show first 10

    # Show when holidays occur
    holiday_days = X_holidays.sum(axis=1) > 0
    print(f"\nNumber of holiday days: {holiday_days.sum()}")
    print(f"First few holiday dates: {X_holidays[holiday_days].index[:5].tolist()}")

except ImportError:
    print("Holiday features not available in this environment.")
    print("To use holiday features, install: pip install holidays")
    print("\nAlternatively, you can create custom calendar features:")

    # Demonstrate manual calendar features
    calendar_features = pd.DataFrame(
        {
            "month": y_train.index.month,
            "quarter": y_train.index.quarter,
            "is_year_end": (y_train.index.month == 12).astype(int),
            "is_year_start": (y_train.index.month == 1).astype(int),
        },
        index=y_train.index,
    )

    print("Manual calendar features:")
    print(calendar_features.head(10))

## 4. Combining Multiple Transformations

Often, you'll want to apply multiple transformations in sequence.

In [ ]:
from sktime.transformations.compose import TransformerPipeline
from sktime.transformations.series.boxcox import BoxCoxTransformer

# Create a pipeline of transformations
transformation_pipeline = TransformerPipeline(
    [
        ("boxcox", BoxCoxTransformer()),  # Variance stabilization
        ("detrend", Detrender()),  # Remove trend
        ("diff", Differencer(lags=1)),  # Remove remaining autocorrelation
    ]
)

# Apply the pipeline
y_transformed = transformation_pipeline.fit_transform(y_train)

# Plot the transformation steps
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.ravel()

# Original
plot_series(y_train, ax=axes[0], title="1. Original Series")

# Box-Cox
boxcox_only = BoxCoxTransformer().fit_transform(y_train)
plot_series(boxcox_only, ax=axes[1], title="2. After Box-Cox")

# Box-Cox + Detrend
boxcox_detrend = TransformerPipeline(
    [("boxcox", BoxCoxTransformer()), ("detrend", Detrender())]
).fit_transform(y_train)
plot_series(boxcox_detrend, ax=axes[2], title="3. After Box-Cox + Detrend")

# Full pipeline
plot_series(y_transformed, ax=axes[3], title="4. After Full Pipeline")

plt.tight_layout()
plt.show()

print("Transformation effects:")
print(f"Original - Mean: {y_train.mean():.2f}, Std: {y_train.std():.2f}")
print(f"Box-Cox - Mean: {boxcox_only.mean():.2f}, Std: {boxcox_only.std():.2f}")
print(f"Final - Mean: {y_transformed.mean():.2f}, Std: {y_transformed.std():.2f}")

## 5. Practical Example: Impact on Forecasting

Let's see how transformations affect forecasting performance.

In [ ]:
from sktime.forecasting.compose import TransformedTargetForecaster

# Forecast without transformation
forecaster_simple = NaiveForecaster(strategy="seasonal_last", sp=12)
forecaster_simple.fit(y_train)
y_pred_simple = forecaster_simple.predict(fh=range(1, 13))

# Forecast with Box-Cox transformation
forecaster_transformed = TransformedTargetForecaster(
    [
        ("boxcox", BoxCoxTransformer()),
        ("forecaster", NaiveForecaster(strategy="seasonal_last", sp=12)),
    ]
)
forecaster_transformed.fit(y_train)
y_pred_transformed = forecaster_transformed.predict(fh=range(1, 13))

# Calculate errors
mape_simple = mean_absolute_percentage_error(y_test, y_pred_simple)
mape_transformed = mean_absolute_percentage_error(y_test, y_pred_transformed)

print(f"MAPE without transformation: {mape_simple:.2%}")
print(f"MAPE with Box-Cox transformation: {mape_transformed:.2%}")
print(f"Improvement: {((mape_simple - mape_transformed) / mape_simple * 100):.1f}%")

# Plot results
plot_series(
    y_train.iloc[-24:],
    y_test,
    y_pred_simple,
    y_pred_transformed,
    labels=["Training", "Actual", "Simple Forecast", "Transformed Forecast"],
    title="Impact of Transformations on Forecasting",
)
plt.legend()
plt.show()

## 6. Best Practices for Transformations

Here are some guidelines for effective use of transformations:

In [ ]:
print("Best Practices for Time Series Transformations:")
print("\n1. VARIANCE STABILIZATION:")
print("   - Use Box-Cox or log transform for series with increasing variance")
print("   - Check if variance changes over time")

# Demonstrate variance check
# Split series into chunks and check variance
chunk_size = len(y_train) // 3
chunks = [
    y_train.iloc[:chunk_size],
    y_train.iloc[chunk_size : 2 * chunk_size],
    y_train.iloc[2 * chunk_size :],
]
print("\n   Variance by period:")
for i, chunk in enumerate(chunks):
    print(f"   Period {i + 1}: {chunk.var():.2f}")

print("\n2. STATIONARITY:")
print("   - Use differencing to remove trends")
print("   - Check ACF/PACF plots after transformation")
print("   - Consider seasonal differencing for seasonal data")

print("\n3. FEATURE ENGINEERING:")
print("   - Fourier features for strong seasonality")
print("   - Calendar features for irregular patterns")
print("   - Domain-specific features when available")

print("\n4. PIPELINE CONSTRUCTION:")
print("   - Order matters: variance stabilization → detrending → differencing")
print("   - Test each step individually")
print("   - Validate on out-of-sample data")

print("\n5. INVERSE TRANSFORMATION:")
print("   - Always ensure transformations are invertible")
print("   - sktime handles inverse transformation automatically")
print("   - Be careful with differencing and missing values")

## 7. Advanced Transformation Example

Let's create a comprehensive transformation pipeline:

In [ ]:
# Create a comprehensive transformation pipeline
comprehensive_pipeline = TransformerPipeline(
    [
        # Step 1: Variance stabilization
        ("boxcox", BoxCoxTransformer(method="mle")),
        # Step 2: Create features before differencing
        ("fourier", FourierFeatures(sp_list=[12], fourier_terms_list=[2])),
        # Step 3: Remove trend and seasonality from target
        ("detrend", Detrender()),
    ]
)

# Apply comprehensive transformation
result = comprehensive_pipeline.fit_transform(y_train)

if hasattr(result, "columns"):  # Check if it's a DataFrame (with features)
    print(f"Transformed data shape: {result.shape}")
    print(f"Features created: {result.columns.tolist()}")

    # Plot original vs transformed target
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    plot_series(y_train, ax=axes[0], title="Original Series")

    # If result is DataFrame, plot the first column (transformed target)
    if hasattr(result, "iloc"):
        plot_series(result.iloc[:, 0], ax=axes[1], title="Transformed Target")

    plt.tight_layout()
    plt.show()
else:
    print(f"Transformed series shape: {result.shape}")

    # Plot comparison
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    plot_series(y_train, ax=axes[0], title="Original Series")
    plot_series(result, ax=axes[1], title="Transformed Series")

    plt.tight_layout()
    plt.show()

## Summary

In this tutorial, you learned:

1. **Transformation Motivation**: Why transformations are important for time series
2. **Target Transformations**: 
   - Differencing for stationarity
   - Detrending for trend removal
   - Box-Cox for variance stabilization
3. **Feature Engineering**:
   - Fourier features for seasonality
   - Holiday/calendar features for special events
4. **Pipeline Construction**: How to combine multiple transformations
5. **Practical Impact**: How transformations improve forecasting performance
6. **Best Practices**: Guidelines for effective transformation use

## Key Takeaways

- **Order Matters**: Apply transformations in logical sequence
- **Validate Impact**: Always test if transformations improve performance
- **Domain Knowledge**: Use domain expertise to guide feature engineering
- **Invertibility**: Ensure transformations can be reversed for interpretable forecasts
- **Automation**: sktime handles inverse transformations automatically

## Next Steps

- Learn about "Pipelines" to integrate transformations with forecasters
- Explore "Cross-validation and Metrics" for robust evaluation
- Try "Hyperparameter Tuning" to optimize transformation parameters